##01

In [ ]:
import heapq
from collections import defaultdict

# Step 1: Create a class for the nodes in the Huffman Tree
class Node:
    def __init__(self, symbol, frequency):
        self.symbol = symbol
        self.frequency = frequency
        self.left = None
        self.right = None

    # Define comparison operators for priority queue (min-heap)
    def __lt__(self, other):
        return self.frequency < other.frequency

# Step 2: Build the Huffman Tree
def build_huffman_tree(frequencies):
    # Initialize a priority queue with leaf nodes
    heap = [Node(symbol, freq) for symbol, freq in frequencies.items()]
    heapq.heapify(heap)

    # Combine nodes until there's only one node left (the root of the tree)
    while len(heap) > 1:
        # Take two nodes with the lowest frequency
        left = heapq.heappop(heap)
        right = heapq.heappop(heap)

        # Combine them into a new node with summed frequency
        merged = Node(None, left.frequency + right.frequency)
        merged.left = left
        merged.right = right

        # Insert the combined node back into the heap
        heapq.heappush(heap, merged)

    # The root of the Huffman Tree
    return heap[0]

# Step 3: Generate Huffman Codes by traversing the Huffman Tree
def generate_codes(node, current_code="", codes={}):
    if node is not None:
        # Check if node is a leaf node
        if node.symbol is not None:
            codes[node.symbol] = current_code
        # Traverse the left and right subtrees
        generate_codes(node.left, current_code + "0", codes)
        generate_codes(node.right, current_code + "1", codes)
    return codes

# Main function to perform Huffman Encoding
def huffman_encoding(symbols_with_frequencies):
    # Step 1: Build the Huffman Tree
    root = build_huffman_tree(symbols_with_frequencies)

    # Step 2: Generate Huffman Codes
    huffman_codes = generate_codes(root)

    return huffman_codes

# Example usage
symbols_with_frequencies = {
    'A': 1/2,
    'B': 1/4,
    'C': 1/8,
    'D':  1/8,
     
}

# Generate Huffman Codes
huffman_codes = huffman_encoding(symbols_with_frequencies)

print("Huffman Codes for the given symbols:")
for symbol, code in huffman_codes.items():
    print(f"{symbol}: {code}")


Huffman Codes for the given symbols:
A: 0
B: 10
C: 110
D: 111


In [ ]:
import heapq
from collections import Counter


def Probabilities(message):
    message = message.upper().replace(" ", "")
    freq = Counter(message)
    P = [fre / len(message) for fre in freq.values()]
    return P, freq


def build_heap(freq):
    heap = [[fre / len(message), [char, ""]] for char, fre in freq.items()]
    heapq.heapify(heap)
    return heap


def build_tree(heap):
    while len(heap) > 1:
        left = heapq.heappop(heap)
        right = heapq.heappop(heap)
        for pair in left[1:]:
            pair[1] = "0" + pair[1]
        for pair in right[1:]:
            pair[1] = "1" + pair[1]
        heapq.heappush(heap, [left[0] + right[0]] + left[1:] + right[1:])
    return heap[0]


message = "aaa bbbbb cccccc dddd ee"
P, freq = Probabilities(message)
print("Probabilities: ", P)
heap = build_heap(freq)
tree = build_tree(heap)
print("Tree:", tree)
for pair in tree[1:]:
    print(pair[0], "->", pair[1])


Probabilities:  [0.15, 0.25, 0.3, 0.2, 0.1]
Tree: [1.0, ['D', '00'], ['B', '01'], ['E', '100'], ['A', '101'], ['C', '11']]
D -> 00
B -> 01
E -> 100
A -> 101
C -> 11


##02

In [40]:
import numpy as np

# Define generator polynomials and message
g0 = [1, 1, 1, 1]
g1 = [1, 1, 0, 1]
message = [1, 0, 1, 0, 1]
K = 4
n = 2

def binary_convolution(msg, generator):
    # Perform binary convolution (mod 2) of message with generator polynomial
    result = []
    for i in range(len(msg) + len(generator) - 1):
        conv_sum = 0
        for j in range(len(generator)):
            if i - j >= 0 and i - j < len(msg):
                conv_sum ^= msg[i - j] * generator[j]  # XOR for modulo-2 addition
        result.append(conv_sum)
    return result

def encode(msg, generators):
    # Encode message using each generator polynomial
    encoded_results = [binary_convolution(msg, g) for g in generators]

    # Interleave bits from each encoded result to form the final encoded message
    max_len = max(len(r) for r in encoded_results)
    for result in encoded_results:
        result.extend([0] * (max_len - len(result)))  # Zero-pad to max length

    encoded_message = [bit for i in range(max_len) for result in encoded_results for bit in [result[i]]]
    return encoded_message

# Define the list of generator polynomials
generators = [g0, g1]
# Encode the message
encoded_message = encode(message, generators)
print('Encoded Message:', encoded_message)


Encoded Message: [1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1]


#003

In [1]:
# Define the input message to be encoded.
message = 'BAABAAABBABBBABBABAAB'

# Initialize an empty dictionary for storing unique substrings with indexes.
dictionary = {}

# Initialize variables:
# - `tmp` for accumulating characters to form substrings,
# - `i` as an index for the dictionary entries,
# - `last` to store the last dictionary index for encoding.
# - `Flag` to track if the current substring is found in the dictionary.
tmp, i, last = '', 1, 0
Flag = True

# Iterate through each character in the message.
for x in message:
    tmp += x  # Add current character to temporary substring.
    Flag = False  # Set flag to indicate substring is not found in dictionary.
    
    # If `tmp` is a new substring, add it to the dictionary.
    if tmp not in dictionary.keys():
        dictionary[tmp] = i  # Store substring with its index.
        tmp = ''  # Reset `tmp` for new substring.
        i += 1  # Increment dictionary index.
        Flag = True  # Set flag to indicate dictionary was updated.

# If there is an unfinished substring, store its dictionary index in `last`.
if not Flag:
    last = dictionary[tmp]

# Initialize `res` with '1' to start encoding.
res = ['1']

# Encode each substring from the dictionary.
for char, idx in list(dictionary.items())[1:]:
    tmp, s = '', ''  # Temporary variables for encoding.
    
    # Process each character in the substring except the last one.
    for x, j in zip(char[:-1], range(len(char))):
        tmp += x
        if tmp in dictionary.keys():  # Find longest prefix in dictionary.
            take = dictionary[tmp]  # Get dictionary index for the prefix.
            s = str(take) + char[j + 1:]  # Concatenate index with remaining character.
    
    # For single-character substrings, directly add to result.
    if len(char) == 1:
        s = char
    res.append(s)

# Append the index of the last substring if it exists.
if last:
    res.append(str(last))

# Mapping of characters 'A' and 'B' to binary values.
mark = {
    'A': 0,
    'B': 1
}

# Convert each encoded value to binary representation.
final_res = []
for x in res:
    tmp = ""
    for char in x:
        if char.isalpha():  # Convert 'A' and 'B' using the `mark` dictionary.
            tmp += bin(mark[char])[2:]
        else:  # Convert numeric indexes to binary.
            tmp += bin(int(char))[2:]
    # Ensure each binary representation is zero-padded to 4 bits.
    final_res.append(tmp.zfill(4))

# Print the final representations.
print(f'N.representatin: {res}')
print("Encoded: ", final_res)


N.representatin: ['1', 'A', '2B', '2A', '3B', '5B', '5A', '1A', '3']
Encoded:  ['0001', '0000', '0101', '0100', '0111', '1011', '1010', '0010', '0011']


##04

In [23]:
def calculate_parity_bits(data_bits):
    # Place data bits in positions 3, 5, 6, and 7
    d3, d5, d6, d7 = map(int, data_bits)
    
    # Calculate parity bits for even parity
    # P1 covers positions 1, 3, 5, 7
    p1 = (d3 + d5 + d7) % 2

    # P2 covers positions 2, 3, 6, 7
    p2 = (d3 + d6 + d7) % 2

    # P4 covers positions 4, 5, 6, 7
    p4 = (d5 + d6 + d7) % 2

    # Return the encoded message
    hamming_code = f"{p1}{p2}{d3}{p4}{d5}{d6}{d7}"
    return hamming_code

def detect_and_correct_error(received_data):
    # Convert the received data to a list of integers for easier manipulation
    received_bits = list(map(int, received_data))

    # Check parity for P1, P2, and P4
    p1_check = (received_bits[0] + received_bits[2] + received_bits[4] + received_bits[6]) % 2
    p2_check = (received_bits[1] + received_bits[2] + received_bits[5] + received_bits[6]) % 2
    p4_check = (received_bits[3] + received_bits[4] + received_bits[5] + received_bits[6]) % 2

    # Calculate the syndrome (error position)
    error_position = p1_check * 1 + p2_check * 2 + p4_check * 4

    if error_position == 0:
        return "No error detected", received_data  # No error
    else:
        # Correct the error at the error_position
        print(f"Error detected at position {error_position}. Correcting it.")
        received_bits[error_position - 1] = 1 - received_bits[error_position - 1]  # Flip the erroneous bit
        return "Error detected and corrected", received_bits

# Example usage
data_bits = "1110"  # Data bits to encode
print(f"Data: {data_bits}")

# Calculate the encoded message (Hamming code)
encoded_data = calculate_parity_bits(data_bits)
print(f"Encoded Data: {encoded_data}")

# Simulate a transmission with an error (let's say bit 6 has an error)
received_data_with_error = "0010110"  # This is the received data with a single-bit error
print(f"Received Data (with error): {received_data_with_error}")

# Detect and correct errors
status, corrected_data = detect_and_correct_error(received_data_with_error)
print(status)
print(f"Corrected Data: {''.join(map(str, corrected_data))}")


Data: 1110
Encoded Data: 0010110
Received Data (with error): 0010110
No error detected
Corrected Data: 0010110


##05

In [3]:
import math

# given
matrix = [[2 / 3, 1 / 3], [1 / 3, 2 / 3]]
print("Symmetric matrix is:")
for i in range(0, 2):
    for j in range(0, 2):
        print('%.2f ' % matrix[i][j], end=' ')
    print()

# Calculate H(Y/X) using formula (1-p)log(1/(1-p))+plog(1/p)
Hp = matrix[0][0] * math.log2(1.0 / matrix[0][0]) + matrix[0][1] * math.log2(1.0 / matrix[0][1])
print("Conditional probability H(Y/X) is = %.3f" % Hp, "bits/msg symbol")

# Now calculate channel capacity using formula C = 1- H(Y/X)
C = 1 - Hp
print("Channel Capacity is = %.3f" % C, "bits/msg symbol")

Symmetric matrix is:
0.67  0.33  
0.33  0.67  
Conditional probability H(Y/X) is = 0.918 bits/msg symbol
Channel Capacity is = 0.082 bits/msg symbol


##06

In [2]:
import heapq
import math
from collections import Counter


def calculate_frequency(my_text):
    my_text = my_text.upper().replace(' ', '')
    frequency = dict(Counter(my_text))
    return frequency


def build_heap(freq):
    heap = [[weight, [char, ""]] for char, weight in freq.items()]
    heapq.heapify(heap)
    return heap


def build_tree(heap):
    while len(heap) > 1:
        lo = heapq.heappop(heap)
        hi = heapq.heappop(heap)
        for pair in lo[1:]:
            pair[1] = '0' + pair[1]
        for pair in hi[1:]:
            pair[1] = '1' + pair[1]
        heapq.heappush(heap, [lo[0] + hi[0]] + lo[1:] + hi[1:])
    return heap[0]


def compute_huffman_avg_length(freq, tree, length):
    huffman_avg_length = 0
    for pair in tree[1:]:
        huffman_avg_length += (len(pair[1]) * (freq[pair[0]] / length))
    return huffman_avg_length


def entropy(freq, length):
    H = 0
    P = [fre / length for _, fre in freq.items()]
    for x in P:
        H += -(x * math.log2(x))
    return H


message = "aaabbbbbccccccddddee"
freq = calculate_frequency(message)
heap = build_heap(freq)
tree = build_tree(heap)
# tree=[20, ['D', '0'], ['B', '01'], ['E', '100'], ['A', '101'], ['C', '11']] not optimal
huffman_avg_length = compute_huffman_avg_length(freq, tree, len(message))
H = entropy(freq, len(message))
print("Huffman : %.2f bits" % huffman_avg_length)
print('Entropy : %.2f bits' % H)
if huffman_avg_length >= H:
    print("Huffman code is optimal")
else:
    print("Code is not optimal")

Huffman : 2.25 bits
Entropy : 2.23 bits
Huffman code is optimal


##07

In [ ]:
import math
from collections import defaultdict

# given
g = defaultdict(list)
xij = [[1, 1, 2], [1, 1], [1, 2, 1], [1, 1]]

def makeGraph(li):
    for node in range(len(li)):
        for x in li[node]:
            g[node].append(x)


def entropy(li):
    H = 0
    for x in li:
        if x == 0:
            continue
        H += -(x * math.log2(x))
    return H


# make graph
makeGraph(xij)
wi = []
for node in range(len(g)):
    wi.append(sum(g[node]))

# we know
# summation(wi)=2w
w = sum(wi) / 2

# the stationary distribution is
# ui=(wi)/2w
ui = [weight / (2 * w) for weight in wi]

# H((wi)/2w)=H(ui)
H_wi_div_2w = entropy(ui)

# H(wij/2*w) = H(g[]/2*w)
wij_div_2w_list = []
for i in range(len(g)):
    wij_div_2w_list += [weight / (2 * w) for weight in g[i]]

# H(wij/2*w) = H(wij_div_2w_list)
H_wij_div_2w = entropy(wij_div_2w_list)

# finally the entropy rate
# H(x)=H(wij/2w)-H(wi/2w)
H_x = H_wij_div_2w - H_wi_div_2w
print('Entropy Rate: %.2f' % H_x)

Entropy Rate: 1.33


##08

In [4]:
import math

matrix = [
    [1 / 8, 1 / 16, 1 / 32, 1 / 32],
    [1 / 16, 1 / 8, 1 / 32, 1 / 32],
    [1 / 16, 1 / 16, 1 / 16, 1 / 16],
    [1 / 4, 0, 0, 0]
]

# the marginal distribution of x
marginal_x = []
for i in range(len(matrix[0])):
    marginal_x.append(sum(matrix[j][i] for j in range(len(matrix))))

# the marginal distribution of y
marginal_y = []
for i in range(len(matrix)):
    marginal_y.append(sum(matrix[i][j] for j in range(len(matrix[0]))))


# H(x)
def entropy(marginal_var):
    H = 0
    for x in marginal_var:
        if x == 0:
            continue
        H += -(x * math.log2(x))
    return H


H_x = entropy(marginal_x)
H_y = entropy(marginal_y)

# conditional entropy
# H(x/y)
H_xy = 0
for i in range(len(matrix)):
    tmp = [(1 / marginal_y[i]) * matrix[i][j] for j in range(len(matrix[0]))]
    H_xy += entropy(tmp) * marginal_y[i]

# H(y/x)
H_yx = 0
for i in range(len(matrix[0])):
    tmp = [(1 / marginal_x[i]) * matrix[j][i] for j in range(len(matrix))]
    H_yx += entropy(tmp) * marginal_x[i]

print('Conditional Entropy H(x|y): ', H_xy)
print('Conditional Entropy H(y|x): ', H_yx)

# Joint entropy
# H(x,y)
H_of_xy = H_x + H_yx
print('Joint Entropy H(x,y): ', H_of_xy)

# Mutual Information
# I(x,y)
I_of_xy = H_y - H_yx
print('Mutual Information: ', I_of_xy)

Conditional Entropy H(x|y):  1.375
Conditional Entropy H(y|x):  1.625
Joint Entropy H(x,y):  3.375
Mutual Information:  0.375
